In [1]:
library(plyr, quietly = TRUE)
library(tidyverse, quietly = TRUE)
library(ggpubr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::desc()      masks plyr::desc()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::id()        masks plyr::id()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::mutate()    masks plyr::mutate()
✖ dplyr::rename()    masks plyr::rename()
✖ dplyr::summarise() masks plyr::summarise()
✖ dplyr::summarize() masks plyr::summarize()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘ggpubr’


The following ob

In [2]:
LLM_score_thresh = 0.8
enrichr_adj_pval_thresh = 0.05
enrichr_JI_thresh = 0.1

In [3]:
LLM_genes_DF = read_delim(file = "data/omics_revamped_LLM_Enrichr_simVals_DF.tsv", delim = "\t") %>%
mutate(enrichr_adj_pVal = `Adjusted P-value`)   


New names:
• `` -> `...1`
Rows: 11310 Columns: 20
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (12): Source, GeneSetID, GeneSetName, GeneList, LLM Name, LLM Analysis, ...
dbl  (8): ...1, Unnamed: 0, n_Genes, Score, Rank, P-value, Adjusted P-value,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
dim(LLM_genes_DF)

[1] 11310    21

In [5]:
get_JI = function(GeneList, enrichr_genes){
    # assume separation is the same
    geneSetGenes = str_split(string = GeneList, pattern = " ")[[1]]
    enrichRGenes = str_split(string = enrichr_genes, pattern = " ")[[1]]
    JI = length(intersect(geneSetGenes, enrichRGenes))/ length(union(geneSetGenes, enrichRGenes))
    
    return(JI)
    }   

In [6]:
## Unified function to filter and select the appropriate row
filter_and_select <- function(df, method = "Best APV") {
    if (method == "Best JI") {
        # used in original revised paper
        filtered <- df %>%
            slice(which.max(enrichr_JI))
    }
    
    if (method == "Best APV") {
        filtered <- df %>%
            filter((enrichr_JI >= enrichr_JI_thresh) & (enrichr_adj_pVal <= enrichr_adj_pval_thresh))
    }
    
    ## For any method if no term meets JI and APV then simply select the one with minimum APV
    if (nrow(filtered) > 0) {
        return(filtered %>% slice(which.min(enrichr_adj_pVal)))
            
    } else {
        return(df %>% slice(which.min(enrichr_adj_pVal)))
    }
}

In [7]:
## GET JI
LLM_genes_DF = LLM_genes_DF %>%
rowwise() %>%
mutate(enrichr_JI = get_JI(GeneList, GO_term_genes))

In [8]:
## Select term based on criteria
LLM_genes_selectedTerms_DF <- LLM_genes_DF %>%
  group_by(Source, GeneSetID, GeneSetName, GeneList) %>%
  do(filter_and_select(.,"Best APV" )) %>%
  ungroup()


In [9]:
dim(LLM_genes_selectedTerms_DF)

[1] 300  22

In [10]:
## Rename column and apply "success" condition
LLM_genes_selectedTerms_DF = LLM_genes_selectedTerms_DF %>%
rowwise() %>%
mutate(
    LLM_success_TF = ifelse(Score >= LLM_score_thresh, TRUE, FALSE),
    enrichr_success_TF = ifelse(!((enrichr_adj_pVal > enrichr_adj_pval_thresh) | (enrichr_JI < enrichr_JI_thresh)), TRUE, FALSE))
        
    

# NOTE: the score will be selected in 6A based on curve

In [11]:
dim(LLM_genes_selectedTerms_DF)

[1] 300  24

In [12]:
write_delim(x = LLM_genes_selectedTerms_DF, file = "data/omics_revamped_LLM_Enrichr_simVals_failure_240625_maxAPV_DF.tsv", delim = "\t")

In [13]:
LLM_genes_selectedTerms_DF = LLM_genes_selectedTerms_DF %>%
mutate(enrichr_success_TF_0.1 = ifelse(((enrichr_adj_pVal > enrichr_adj_pval_thresh) | (enrichr_JI < 0.1)), FALSE, TRUE))

In [14]:
table(LLM_genes_selectedTerms_DF[,c("enrichr_success_TF_0.1", "LLM_success_TF")])


                      LLM_success_TF
enrichr_success_TF_0.1 FALSE TRUE
                 FALSE   163   97
                 TRUE      4   36

In [ ]:
LLM_success_TF      True  False  Total
enrichr_success_TF                    
True                  36      4     40
False                 97    163    260
Total                133    167    300